In [ ]:
#| default_exp markdown.obsidian.personal.machine_learning.tokenize

# markdown.obsidian.personal.machine_learning.tokenize
> Functions for gathering and processing tokenization data and for using ML models trained with such data.

Previous, `trouver` just had functionalities for using ML models to identify newly introduced notations in text and for gathering data to train such models. Moreover, such models were merely classification models, and using these models to identify newly introduced notations had a lot of computational redundancies.

This module aims to provide the same functionalities for both definitions and notations by training and using token classification models instead.

In [ ]:
#| export
import os 
from os import PathLike
from pathlib import Path
from typing import Union

import bs4
import pandas as pd
from transformers import BatchEncoding, PreTrainedTokenizer, PreTrainedTokenizerFast

from trouver.helper import current_time_formatted_to_minutes, definition_asterisk_indices, double_asterisk_indices, notation_asterisk_indices, replace_string_by_indices, remove_html_tags_in_text
from trouver.markdown.markdown.file import MarkdownFile
from trouver.markdown.obsidian.personal.note_processing import process_standard_information_note
from trouver.markdown.obsidian.vault import VaultNote

In [ ]:
from unittest import mock

from datasets import ClassLabel, Dataset, Features, Sequence, Value
from transformers import AutoTokenizer
from fastcore.test import *


## Gather ML data from information notes

In [ ]:
#| export
def convert_double_asterisks_to_html_tags(
        text: str
        ) -> str:
    """
    Replace the double asterisks, which signify definitions and notations,
    in `text` with HTML tags.
    """
    double_asts = double_asterisk_indices(text)
    replacement_html_tags = [
        _html_tag_from_double_ast(text[start:end])
        for start, end in double_asts]
    return replace_string_by_indices(
        text, double_asts, replacement_html_tags)


def _html_tag_from_double_ast(
        double_ast_string: str # Starts and ends with double asts
        ) -> str:
    """
    Get the HTML tag representing definition or notation data from
    a string surrounded by double asterisks.

    This is used in the `_convert_double_asterisks_to_html_tags` function.
    """
    no_asts = double_ast_string[2:-2]
    if notation_asterisk_indices(double_ast_string):
        return f'<span notation="">{no_asts}</span>'
    else:
        return f'<b definition="">{no_asts}</b>'


In [ ]:
print(convert_double_asterisks_to_html_tags("**hi**. Here is a notation **$asdf$**"))
test_eq(convert_double_asterisks_to_html_tags("**hi**. Here is a notation **$asdf$**"), '<b definition="">hi</b>. Here is a notation <span notation="">$asdf$</span>')


<b definition="">hi</b>. Here is a notation <span notation="">$asdf$</span>


In [ ]:
#| export
def raw_text_with_html_tags_from_markdownfile(
        mf: MarkdownFile,
        vault: PathLike
        ) -> str:
    """
    Process the `MarkdownFile`, replacing the double asterisk surrounded
    text indicating definitions and notations to be HTML tags instead.
    """
    mf = process_standard_information_note(
        mf, vault, remove_double_asterisks=False,
        remove_html_tags=False)
    return convert_double_asterisks_to_html_tags(str(mf))



In [ ]:
#| hide

# TODO: 
# I want to make sure that footnotes are getting properly removed.
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Something  

Some kind of potato[^2]

[^2]: Some footnote

[[link_to_note|Some link]]


# See Also
# Meta
## References and Citations
""") 
raw_text_with_html_tags_from_markdownfile(mf, None)


'Some kind of potato[^2]\n\n[^2]: Some footnote\n\nSome link\n'

In [ ]:
#| hide
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
""")

In the following example, let `mf` be the following `MarkdownFile`:

In [ ]:
print(str(mf))

---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations


The `raw_text_with_html_tags_from_markdownfile` function processes the `MarkdownFile` much in the same way as the `process_standard_information_note` function, except it 1. preserves HTML tags, and 2. replaces text surrounded by double asterisks `**` with HTML tags signifiying whether the text displays a definition or a notation.

In the below example, note that the `vault` parameter is set to `None`; this is fine for this example becaues the `process_standard_information_note` function only needs a `vault` argument when embedded links need to be replaced with text (via the `MarkdownFile.replace_embedded_links_with_text` function), but `mf` has no embedded links.

In [ ]:
print(raw_text_with_html_tags_from_markdownfile(mf, None))

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its <b definition="">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span>



In [ ]:
#| hide
assert '**' not in raw_text_with_html_tags_from_markdownfile(mf, None)

In [ ]:
#| export
# TODO: implement a measure to not get the definition identification data, e.g. by 
# detecting a `_auto/definition_identification` tag.
def html_data_from_note(
        note: VaultNote,
        vault: PathLike
        ) -> Union[dict, None]: # The keys to the dict are "Note name", "Raw text", "Tag data". However, `None` is returned if `note` does not exist or the note is marked with auto-generated, unverified data.
    # TODO: implement obtaining multiple datapoints from a single note
    # Via typos for example.
    """Obtain html data for token classification from the information note.

    Currently, the token types mainly revolve around definitions and
    notations.

    If `note` has the tag `_auto/def_and_notat_identified`, then the data
    in the note is assumed to be auto-generated and not verified and
    `None` is returned.

    **Returns**
    - Union[dict, None]
        - The keys-value pairs are 
            - `"Note name"` - The name of the note
            - `"Raw text"` - The raw text to include in the data.
            - `"Tag data"` - The list with HTML tags carrying definition/notation
              data and their locations in the Raw text. See the second output to
              the function `remove_html_tags_in_text`.
                - Each element of the list is a tuple consisting of a ``bs4.element.Tag``
                  and two ints.
    """
    if not note.exists():
        return None
    mf = MarkdownFile.from_vault_note(note)
    if mf.has_tag('_auto/def_and_notat_identified'):
        return None
    raw_text_with_tags = raw_text_with_html_tags_from_markdownfile(mf, vault)
    raw_text, tags_and_locations = remove_html_tags_in_text(raw_text_with_tags)
    return {
        "Note name": note.name,
        "Raw text": raw_text,
        "Tag data": tags_and_locations}

In the following example, we mock a `VaultNote` whose content is that of `mf` in the example for the `raw_text_with_html_tags_from_markdownfile` function.

In [ ]:
with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__.MarkdownFile.from_vault_note') as mock_from_vault_note):
    mock_VaultNote.exists.return_value = True
    mock_VaultNote.name = "Note's name"
    mock_from_vault_note.return_value = mf

    html_data = html_data_from_note(mock_VaultNote, None)
    print(html_data)


{'Note name': "Note's name", 'Raw text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that\n$L = \\varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its Galois group $\\operatorname{Gal}(L/K)$\n', 'Tag data': [(<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, 64, 76), (<span notation="">$\operatorname{Gal}(L/K)$</span>, 77, 102), (<b definition="">Galois group</b>, 330, 342), (<span notation="">$\operatorname{Gal}(L/K)$</span>, 343, 368)]}


In [ ]:
#| export
def tokenize_html_data(
        html_locus: dict, # An output of `html_data_from_note`
        tokenizer: Union[PreTrainedTokenizer, PreTrainedTokenizerFast],
        max_length: int, # Max length for each sequence of tokens
        ner_tag_from_html_tag: callable, # takes in a bs4.element.Tag and outputs the ner_tag (as a string or `None`)
        label2id: dict[str, int], # The keys ner_tag's of the form f"I-{output}" or f"B-{output}" where `output` is an output of `ner_tag_from_html_tag`.
        default_label: str = "O", # The default label for the NER tagging.
        ) -> tuple[list[list[str]], list[list[int]]]: # The first list consists of the tokens and the second list consists of the named entity recognition tags.
    """Actually tokenize the html data outputted by `html_data_from_note`.

    To account for the possibility that the raw text is long,
    this function uses the `tokenizer.batch_encode_plus` function
    to tokenize the text into sequences. 
    """
    tokenized = tokenizer.batch_encode_plus(
        [html_locus["Raw text"]], max_length=max_length, return_overflowing_tokens=True,
        return_offsets_mapping=True, truncation=True)

    default_id = label2id[default_label]        
    ner_ids = [[default_id for _ in seq_input_ids]
               for seq_input_ids in tokenized['input_ids']]
    for tag, start, end in html_locus["Tag data"]:
        ner_tag = ner_tag_from_html_tag(tag)
        if ner_tag is None:
            continue  # `ner_tag` is not of relevant data.
        tuppy = _start_end_seqs_indices_for_html_tag(tokenized, start, end - 1)
        (start_seq, start_index_in_seq), (end_seq, end_index_in_seq) = tuppy
        _set_ner_ids_for_tag(
            ner_ids, start_seq, start_index_in_seq, end_seq, end_index_in_seq,
            label2id, ner_tag)
    return tokenized["input_ids"], ner_ids


def _start_end_seqs_indices_for_html_tag(
        tokenized: BatchEncoding,
        tag_start_ind: int,
        tag_end_ind: int
        ) -> tuple[tuple[int, int], tuple[int, int]]: # The first tuple is `(a, b)` where `tokenized['input_ids'][a][b]` is the token corresponding to the start of the HTML tag's (raw) text. The second tuple is `(c, d)` where `tokenized['input_ids'][c][d]` is the token corresponding to the end of the HTML tag's (raw) text.
    start_seq = _search_seq_ind_for_char(tokenized['offset_mapping'], tag_start_ind)
    # start_index_in_seq = tokenized.char_to_token(batch_or_char_index=start_seq, char_index=tag_start_ind)
    start_index_in_seq = _search_within_seq_for_char(tokenized['offset_mapping'][start_seq], tag_start_ind)
    end_seq = _search_seq_ind_for_char(tokenized['offset_mapping'], tag_end_ind)
    # end_index_in_seq = tokenized.char_to_token(batch_or_char_index=end_seq, char_index=tag_end_ind)
    end_index_in_seq = _search_within_seq_for_char(tokenized['offset_mapping'][end_seq], tag_end_ind)
    return (start_seq, start_index_in_seq), (end_seq, end_index_in_seq)


def _min_max_char_ind_for_seq(
        offset_for_seq: list[tuple[int,int]] # An item in tokenized['offset_mapping']
        ):
    min_char_ind, max_char_ind = 0, 0
    for inds in offset_for_seq:
        if inds != (0,0):
            min_char_ind = inds[0]
            break
    for inds in reversed(offset_for_seq):
        if inds != (0,0):
            max_char_ind = inds[1]
            break
    return min_char_ind, max_char_ind

def _char_is_in_seq(
        offset_for_seq: list[int], # An item in tokenized['offset_mapping']
        char: int # The index of a character in the original raw text
        ) -> bool:
    min_char_ind, max_char_ind = _min_max_char_ind_for_seq(offset_for_seq)
    return min_char_ind <= char and char < max_char_ind

def _search_seq_ind_for_char(
        offsets: list[tuple[int, int]], # tokenized['offset_mapping']
        char: int # The index of a character in the original raw text
        ) -> int:
    """
    Binary search the index of the sequence containing the token at the 
    location of the index `char` within the original (raw) text.

    Based on pseudocode from https://pseudoeditor.com/guides/binary-search
    """
    left = 0
    right = len(offsets) - 1
    while left <= right:
        mid = (left + right) // 2
        min_char_ind, max_char_ind = _min_max_char_ind_for_seq(offsets[mid])
        if min_char_ind <= char and char < max_char_ind:
            return mid
        elif max_char_ind <= char:
            left = mid + 1
        else:
            right = mid - 1
    return -1  # This should not be returned under normal use.


def _search_within_seq_for_char(
        seq_offset: list[tuple[int, int]],
        char: int
    ) -> int:
    """
    Binary search for the index within the sequence corresponding
    to the token at the location of the index `char` within the
    original (raw) text.

    Based on pseudocode from https://pseudoeditor.com/guides/binary-search
    """
    left = 0
    right = len(seq_offset) - 1
    while left <= right:
        mid = (left + right) // 2
        min_char_ind, max_char_ind = seq_offset[mid] 
        if min_char_ind <= char and char < max_char_ind:
            return mid
        elif max_char_ind <= char:
            left = mid + 1
        else:
            right = mid - 1
    return -1  # This should not be returned under normal use.


def _set_ner_ids_for_tag(
        ner_ids: list[list[int]],
        start_seq: int, 
        start_index_in_seq: int,
        end_seq: int,
        end_index_in_seq: int,
        label2id: dict[str, int],
        ner_tag: str
        ) -> None:
    """
    After the locations of the tokens corresponding to a HTML tag have been found, 
    mark within `ner_ids` the appropriate NER tags at the locations corresponding
    to the tokens' locations.
    """
    ner_ids[start_seq][start_index_in_seq] = label2id[f"B-{ner_tag}"]
    i_ner_id = label2id[f"I-{ner_tag}"]
    seq, ind = start_seq, start_index_in_seq + 1
    while seq < end_seq or ind <= end_index_in_seq:
        if len(ner_ids[seq]) <= ind:
            seq += 1
            ind = 0
        else:
            ner_ids[seq][ind] = i_ner_id 
            ind += 1
    


def def_or_notat_from_html_tag(
        tag: bs4.element.Tag
        ) -> Union[str, None]:
    """
    Can be passed as the `ner_tag_from_html_tag` argument in `tokenize_html_data`
    for the purposes of compiling a dataset for definition and notation
    identification.

    The strings f"I-{output}" and f"B-{output}" are valid ner_tags. To use for 
    """
    if "definition" in tag.attrs:
        return "definition"
    elif "notation" in tag.attrs:
        return "notation"
    return None  # If the HTML tag carries neither definition nor notation data.

In [ ]:
#| hide
test_eq(_min_max_char_ind_for_seq([(0,0), (1,3), (3,4), (4,7), (7,15), (0,0)]), (1,15))

offsets = [[(0,0), (0,3), (4,5), (5,6), (6,7), (7,8), (8,9),],
           [(10,12), (13,14), (15,18), (18,24)],
           [(25,28), (29,35), (36,42), ]]
test_eq(_search_seq_ind_for_char(offsets, 0), 0)
test_eq(_search_seq_ind_for_char(offsets, 1), 0)
test_eq(_search_seq_ind_for_char(offsets, 5), 0)
test_eq(_search_seq_ind_for_char(offsets, 8), 0)
# I don't think that character index 9 is something that I need to worry about.
test_eq(_search_seq_ind_for_char(offsets, 10), 1)
test_eq(_search_seq_ind_for_char(offsets, 23), 1)
test_eq(_search_seq_ind_for_char(offsets, 25), 2)
test_eq(_search_seq_ind_for_char(offsets, 41), 2)

We continue with an example using the HTML data from the example for the `html_data_from_note` function.

In [ ]:
html_data['Raw text']
html_data["Tag data"]

[(<b definition="Galois group of a separable and normal finite field extension">Galois group</b>,
  64,
  76),
 (<span notation="">$\operatorname{Gal}(L/K)$</span>, 77, 102),
 (<b definition="">Galois group</b>, 330, 342),
 (<span notation="">$\operatorname{Gal}(L/K)$</span>, 343, 368)]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
label2id = {
    "O": 0,
    "B-definition": 1,
    "I-definition": 2,
    "B-notation": 3,
    "I-notation": 4
}
token_ids, ner_tag_ids = tokenize_html_data(html_data, tokenizer, 510, def_or_notat_from_html_tag, label2id)

For this example, `max_length` is set to 510 (tokens). The string ("Raw text") is not very long, so only one sequence should be present.

In [ ]:
test_eq(len(token_ids), 1)
test_eq(len(ner_tag_ids), 1)

Now let us see what has been tagged:

In [ ]:
id2label = {value: key for key, value in label2id.items()}
id2label

{0: 'O',
 1: 'B-definition',
 2: 'I-definition',
 3: 'B-notation',
 4: 'I-notation'}

In [ ]:
for token_id, ner_tag in zip(token_ids[0], ner_tag_ids[0]):
    if ner_tag != 0:
        print(f"{tokenizer.decode(token_id)}\t\t{id2label[ner_tag]}")

gal		B-definition
##ois		I-definition
group		I-definition
$		B-notation
\		I-notation
operator		I-notation
##name		I-notation
{		I-notation
gal		I-notation
}		I-notation
(		I-notation
l		I-notation
/		I-notation
k		I-notation
)		I-notation
$		I-notation
gal		B-definition
##ois		I-definition
group		I-definition
$		B-notation
\		I-notation
operator		I-notation
##name		I-notation
{		I-notation
gal		I-notation
}		I-notation
(		I-notation
l		I-notation
/		I-notation
k		I-notation
)		I-notation
$		I-notation


Let us set `max_length` to be shorter to observe an example of a tokenization of a single text across multiple sequences (Of course, in practice, the max token length would be set to be longer, say around 512 or 1024.):

In [ ]:
token_ids, ner_tag_ids = tokenize_html_data(html_data, tokenizer, 20, def_or_notat_from_html_tag, label2id)

In [ ]:
print(len(token_ids))
print(len(ner_tag_ids))

7
7


In [ ]:
ner_tag_ids

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 [2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 3, 4, 4],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0]]

## Gathering data 

The following is sample code to then gather data for definition/notation identification

In [ ]:
#| notest
notes = [] # Replace with actual notes
vault = '' # Replace with actual vault

html_data = [html_data_from_note(note, vault) for note in notes]
max_length = 1022

tokenized_html_data = [tokenize_html_data(html_locus, tokenizer, max_length, def_or_notat_from_html_tag, label2id) for html_locus in html_data]
token_id_data = [token_ids for token_ids, _ in tokenized_html_data]
ner_tag_data = [ner_tag_ids for _, ner_tag_ids in tokenized_html_data]
token_seqs = [token_seq for token_seq in token_ids for token_ids in token_id_data]
ner_tag_seqs = [ner_tag_seq for ner_tag_seq in ner_tag_ids for ner_tag_ids in ner_tag_data]

In [ ]:
#| notest
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
max_length = 1022
label2id = {
    "O": 0,
    "B-definition": 1,
    "I-definition": 2,
    "B-notation": 3,
    "I-notation": 4
} 
id2label = {value: key for key, value in label2id.items()}

In [ ]:
#| notest
note_names, token_seqs, ner_tag_seqs = [], [], []
for html_locus, (token_ids, ner_tag_ids) in zip(html_data, tokenized_html_data):
    note_names.extend([html_locus["Note name"]] * len(token_ids))
    token_seqs.extend(token_ids)
    ner_tag_seqs.extend(ner_tag_ids)

In [ ]:
#| notest
ner_tags = ClassLabel(names=list(label2id))

ds = Dataset.from_dict(
        {"note name": note_names,
        "tokens": token_ids,
        "ner_tags": ner_tag_ids},
        features=Features(
            {"tokens": Sequence(Value(dtype='string')), "ner_tags": Sequence(ner_tags)}
        ))

ds.save_to_disk(".")

# ds.load_from_disk(".")
    

## Use the trained model

See https://huggingface.co/docs/transformers/tasks/token_classification for training a token classification model.